In [2]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun
import OLS_Estimation as OLS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_csv('Dataset.csv')
Nobs=df['ID'].count()
T = df['Year'].unique().size
df['const']=np.ones((Nobs,1))
df['HP**2'] = df['HP']**2
df['Range**2'] = df['Range']**2
df['Chargetime**2'] = df['Chargetime']**2
#Skalér for bedre fortolkning

df['Price'] = df['Price']/10_000 #(Ændring i ms(%) for en ændring i pris på 10.000)
df['HP'] = df['HP']/10           #(Ændring i ms(%) for en ændring i HP på 10)
df['Range'] = df['Range']/10     #(Ændring i ms(%) for en ændring i rækkevidde på 10)

In [4]:
estimation = OLS.get_coefficients_OLS(df)


model_labels = df['Model'].unique()
#j = 0
#for i in model_labels:
#    j = j
#    print(f'Car {j}: {i}')
#    j = j+1  

repeated_models = np.repeat(model_labels, T)
print(f' models: {len(model_labels)} \n repeated models: {len(repeated_models)}')
      
print(f'\n \n Coefficients:\n{estimation} \ntype: {type(estimation)} \n shape: {estimation.shape} \n\n')



 models: 107 
 repeated models: 1177

 
 Coefficients:
const        -3.253125
Price        -0.009158
Range         0.089713
HP           -0.024147
Chargetime   -0.041158
Segment_B    -0.382345
Segment_C     0.234436
Segment_D     0.094869
Segment_E    -1.258709
Segment_F    -1.109602
Segment_M    -1.514417
Segment_J    -1.159095
Year_2014     0.473700
Year_2015    -0.364670
Year_2016     0.101828
Year_2017    -0.107238
Year_2018    -0.709046
Year_2019    -1.140337
Year_2020    -1.853072
Year_2021    -2.850731
Year_2022    -3.312345
Year_2023    -3.987426
China         0.000000
dtype: float64 
type: <class 'pandas.core.series.Series'> 
 shape: (23,) 




In [5]:
#Manipuler for ms = 0 ikke har ssh for at blive væltet.
df_calc = df.copy()
df_calc = pd.get_dummies(df_calc, columns=['Segment'], drop_first=True)
df_calc = pd.get_dummies(df_calc, columns=['Year'], drop_first=True)
df_calc['China'] = (df_calc['Country'] == 'CH').astype(int)

df_calc = fun.straf_0ms(df_calc)

df_dummies = df.copy()
df_dummies = pd.get_dummies(df_dummies, columns=['Segment'], drop_first=True)
df_dummies = pd.get_dummies(df_dummies, columns=['Year'], drop_first=True)
df_dummies['China'] = (df_dummies['Country'] == 'CH').astype(int)


print(f' df_calc shape: {df_calc.shape} \n df_dummies shape: {df_dummies.shape}')


c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 10_000_000


 df_calc shape: (1177, 33) 
 df_dummies shape: (1177, 33)


In [6]:
x = df_calc[['const', 'Price', 'Range', 'HP', 'Chargetime']]
dummies = df_calc[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)

columns = X.columns
print(f' columns: {columns}')

X = np.array(X)
print(f' X shape: {X[:,2:].shape}\n X type {type(X)}\n')

p_j = df_calc['Price']
p_j = np.array(p_j)
print(f' p_j shape: {p_j.shape} \n p_j type: {type(p_j)}\n ')

alpha = estimation['Price']
alpha = np.array(alpha)
beta = estimation
beta = np.array(beta)
print(f' alpha {alpha} \n beta shape {beta.shape}')


 columns: Index(['const', 'Price', 'Range', 'HP', 'Chargetime', 'Segment_B', 'Segment_C',
       'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
       'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018',
       'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
       'China'],
      dtype='object')
 X shape: (1177, 21)
 X type <class 'numpy.ndarray'>

 p_j shape: (1177,) 
 p_j type: <class 'numpy.ndarray'>
 
 alpha -0.009158424574739511 
 beta shape (23,)


In [7]:
import numpy as np
ccp_list = fun.ccp(alpha, beta, X, p_j)

true_ccp = []
for ccp in ccp_list:
    if ccp > 0.0000001:
        true_ccp.append(ccp)
    else:
        continue
print(f' len : {len(true_ccp)}')


share_j: 1177
 choice probability sum: 1.0 
 ccp:[array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.00218445]]), array([[0.00134063]]), array([[0.00071396]])]
 len : 334


In [21]:
df['cost'] = fun.cost(p_j, ccp_list, alpha)
filtered_df = df[(df['cost'] >= 0.0001) & (df['cost'] <= 1_000_000)]
sorted_df = filtered_df.sort_values(by='cost', ascending=False)
sorted_df

c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:163: RuntimeWarning: divide by zero encountered in divide
  cost[i] = p_j[i] - (alpha/s[i])


,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,HP**2,Range**2,Chargetime**2,cost
956,167,2023,0.000048,Smart,Fortwo,12.7,21.045750,8.0,60,Hatchback,A,DE,3,1.0,6400,16129,3600,410.063472
516,99,2023,0.000145,Maxus,Euniq5,28.9,37.209879,17.4,45,MPV,M,CN,9,1.0,30276,83521,2025,361.725558
637,114,2023,0.001503,Mercedes-Benz,EQV,34.6,76.405300,20.1,41,MPV,M,DE,93,1.0,40401,119716,1681,328.675524
197,21,2023,0.000081,BYD,Tang,40.0,56.015450,50.9,45,SUV,E,CN,5,1.0,259081,160000,2025,303.650352
846,142,2023,0.000048,Peugeot,Expert,31.5,55.608733,13.4,38,MPV,M,FR,3,1.0,17956,99225,1444,262.644035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,187,2015,0.020041,Volkswagen,up!,25.6,17.376192,8.1,48,Hatchback,A,DE,79,1.0,6561,65536,2304,19.308041
1122,187,2013,0.067114,Volkswagen,up!,25.6,17.376192,8.1,48,Hatchback,A,DE,30,1.0,6561,65536,2304,18.717718
902,158,2013,0.205817,Renault,Zoe,36.5,17.315000,13.4,56,Hatchback,B,FR,92,1.0,17956,133225,3136,18.482617
1123,187,2014,0.147475,Volkswagen,up!,25.6,17.376192,8.1,48,Hatchback,A,DE,219,1.0,6561,65536,2304,18.211552


In [52]:

model_probability = pd.DataFrame({'Model': repeated_models, 'CCP': ccp_list})

filtered_df = model_probability[model_probability['CCP'] > 0.00001]

#print(f'shape: {model_probability.shape}\nmodel_probability:\n{model_probability}')
print(f'shape: {filtered_df.shape}\nfiltered_df:\n{filtered_df}')


models = filtered_df['Model']
models = np.array(models)
ccps = filtered_df['CCP']
ccps = np.array(ccps)
print(f' models: {len(models)} \n ccps: {len(ccps)}')

print(type(ccps[0]))


shape: (334, 2)
filtered_df:
       Model                        CCP
8         U5  [[0.0021844505603098585]]
9         U5   [[0.001340625403939621]]
10        U5  [[0.0007139624927102292]]
21        U6  [[0.0006626658733684756]]
28    e-tron   [[0.001650690726433547]]
...      ...                        ...
1154    EX30  [[0.0007477520532243017]]
1163    XC40   [[0.002439572937912101]]
1164    XC40  [[0.0016034417392778734]]
1165    XC40  [[0.0007993176907465156]]
1176    Free  [[6.748747585148109e-05]]

[334 rows x 2 columns]
 models: 334 
 ccps: 334
<class 'numpy.ndarray'>


In [53]:
probability_ratio = fun.probability_ratio(ccps, models)
print(f'shape: {probability_ratio.shape} \n probability_ratio: {probability_ratio}')

shape: (334, 334) 
 probability_ratio:                              U5                        U5  \
U5                      [[1.0]]    [[1.6294265004158028]]   
U5       [[0.6137128613931444]]                   [[1.0]]   
U5      [[0.32683847631184454]]    [[0.5325592746580421]]   
U6      [[0.30335585771942497]]   [[0.49429607362439687]]   
e-tron   [[0.7556548801907428]]    [[1.2312840869513246]]   
...                         ...                       ...   
EX30    [[0.34230669570211514]]    [[0.5577636012467946]]   
XC40     [[1.1167901815850911]]     [[1.819727517278924]]   
XC40     [[0.7340251907786045]]    [[1.1960400978274235]]   
XC40       [[0.36591246571089]]     [[0.596227468461813]]   
Free    [[0.03089448535832652]]  [[0.050340293159565244]]   

                             U5                       U6  \
U5        [[3.059615291578693]]   [[3.2964585141616216]]   
U5       [[1.8777252553569797]]    [[2.023078987189922]]   
U5                      [[1.0]]   [[1.0774094779

In [10]:
marginal_effects = fun.marginal_effects(ccp_list, repeated_models, columns, estimation)
print(f'shape: {marginal_effects.shape} \n marginal_effects: {marginal_effects}')

shape: (1177, 23) 
 marginal_effects:                            const                      Price  \
U5                      [[-0.0]]                    [[0.0]]   
U5                      [[-0.0]]                    [[0.0]]   
U5                      [[-0.0]]                    [[0.0]]   
U5                      [[-0.0]]                    [[0.0]]   
U5                      [[-0.0]]                    [[0.0]]   
...                          ...                        ...   
Free                    [[-0.0]]                    [[0.0]]   
Free                    [[-0.0]]                    [[0.0]]   
Free                    [[-0.0]]                    [[0.0]]   
Free                    [[-0.0]]                    [[0.0]]   
Free  [[-6.180789573253437e-07]]  [[6.054505528684722e-06]]   

                            Range                          HP  \
U5                       [[-0.0]]                    [[-0.0]]   
U5                       [[-0.0]]                    [[-0.0]]   
U5        

In [11]:
cross_marginal_effects = fun.cross_marginal_effects(ccp_list, estimation)
print(f'shape: {cross_marginal_effects.shape} \n cross_marginal_effects: {cross_marginal_effects}')

KeyboardInterrupt: 

In [ ]:
elasticity = fun.elasticity(ccp_list, repeated_models, columns, estimation, X)
print(f'shape: {elasticity.shape} \n elasticity: {elasticity}')

elasticity shape: 
(1177, 23)
shape: (1177, 23) 
 elasticity:                         const                   Price  \
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
...                       ...                     ...   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free  [[-1869.3367521777004]]  [[3.3227650848306123]]   

                         Range                       HP Chargetime  \
U5    [[-0.37035523076277393]]  [[-1.1722891235561768]]   [[-0.0]]   
U5    [[-0.37035523076277393]]  [[-1.1722891235561768]]   [[-0.0]]   
U5    [[-0.37035523076277393]]  [[-1.172289

In [ ]:
cross_elasticity = fun.cross_elasticity(ccp_list, estimation, X, repeated_models)

1177
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [